## import libraries

In [ ]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 31.5 MB 58.9 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import cv2 as cv
import numpy as np
import mediapipe as mp
import math
import time
from google.colab.patches import cv2_imshow
from IPython.display import clear_output
from PIL import Image
import matplotlib.pyplot as plt
import os 
from numpy.lib.function_base import average


## initialize variables

In [ ]:
# The position of the center of the iris in relation to the eye
storage_cases = {
    "center": 0 ,
    "right": 0 , 
    "left":0 ,
    "up":0,
    "down":0,
    "right _ up":0,
    "right _ down":0,
    "left _ up":0,
    "left _ down":0
}

In [ ]:
# some of points of face mesh used in code 

LEFT_EYE = [362,382,381,380,374,373,390,249,263,466,388,387,386,385,384,398]
RIGHT_EYE = [33,7,163,144,145,153,154,155,133,173,157,158,159,160,161,246]
# points of iris
RIGHT_IRIS = [474,475, 476, 477]
LEFT_IRIS = [469, 470, 471, 472]

L_H_LEFT = [33]  # right eye right most landmark
L_H_RIGHT = [133]  # right eye left most landmark
R_H_LEFT = [362]  # left eye right most landmark
R_H_RIGHT = [263]  # left eye left most landmark

R_U_landmark = [386] #right eye up most landmark
R_D_landmark = [374] #right eye down most landmark
L_U_landmark = [159] #left eye up most landmark
L_D_landmark = [145] #left eye down most landmark

## some of methods

In [ ]:
def euclidean_distance(point1,point2):
  """
  calculate euclidean distance between two points
  input: point1, point2 : x and y for each point
  output: distance between point1 and point2 
  
  """
  x1,y1 = point1.ravel()
  x2,y2 = point2.ravel()
  distance = math.sqrt((x2-x1)**2 + (y2-y1)**2)
  return distance

In [ ]:
# thresholds reprecent ratio of distance between center of iris and point
right_thre = 0.42
left_thre = 0.57
up_thre = 0.25
down_thre=0.58

def iris_position_in_eye(iris_center,right_point ,left_point , up_point ,down_point):
    """
    iris position :
    is return position of iris center and ratio_horizontal,ratio_vertical
    """
  
    center_to_right_dist = euclidean_distance(iris_center,right_point)
    center_to_up_dist = euclidean_distance(iris_center,up_point)
    
    total_distance_horizontal = euclidean_distance(right_point,left_point)
    total_distance_vertical = euclidean_distance(up_point,down_point)
    
    ratio_horizontal = center_to_right_dist/total_distance_horizontal

    if total_distance_vertical==0:
        ratio_vertical =0.1
    else:
        ratio_vertical = center_to_up_dist/total_distance_vertical
    
    
    iris_position = ""
    #iris in center horizontal 
    if ratio_horizontal > right_thre and ratio_horizontal <=left_thre:
        if ratio_vertical <= up_thre:
            storage_cases["up"]+=1
            iris_position ="up"
        elif ratio_vertical > up_thre and ratio_vertical <= down_thre:
            storage_cases["center"]+=1
            iris_position = "center"
        else:
            storage_cases["down"]+=1
            iris_position = "down"
        
    #iris in center vertical
    elif ratio_vertical > up_thre and ratio_vertical <= down_thre:
        if ratio_horizontal <= right_thre:
            storage_cases["right"]+=1
            iris_position ="right"
        elif ratio_horizontal > right_thre and ratio_horizontal <=left_thre:
            storage_cases["center"]+=1
            iris_position = "center"
        else:
            storage_cases["left"]+=1
            iris_position = "left"
            
    # iris in right horizontal    
    elif ratio_horizontal <= right_thre:
        if ratio_vertical <= up_thre:
            storage_cases["right _ up"]+=1
            iris_position ="right _ up"
        elif ratio_vertical > down_thre:
            storage_cases["right _ down"]+=1
            iris_position ="right _ down"
            
    # iris in left horizontal 
    else:
        if ratio_vertical <= up_thre:
            storage_cases["left _ up"]+=1
            iris_position ="left _ up"
        elif ratio_vertical > down_thre:
            storage_cases["left _ down"]+=1
            iris_position ="left _ down"
            
    return iris_position,ratio_horizontal,ratio_vertical

In [ ]:
def normaliz_pixel( normalized_x, normalized_y, image_width, image_height):
  """
  input: 
  x ,y :  both normalized to [0.0, 1.0] by the image width and the image height 
  image_width, image_height: represent width and height of the image

  output:
  x , y: represent point in the image by pixels
  """
  x_px = min(math.floor(normalized_x * image_width), image_width - 1)
  y_px = min(math.floor(normalized_y * image_height), image_height - 1)
  return x_px, y_px

## The main method : detect_eye_contact_unsupervised

In [ ]:
def detect_eye_contact_unsupervised (video_path , save_text = True , vis = True , display = False,
         output_path = '/content/drive/MyDrive/The results of eye contact/unsupervised'):
  """
  input: 
  video_path: place of video 
  save_text: ture or false 
  vis: true or false 
  output_path: Where to store results 
  output: 
  confidense , 
  0 if presenter without eye contact 
  or 1 if presenter with eye contact 
  """
  # face detection
  mp_face_detection = mp.solutions.face_detection
  # face mesh 
  mp_face_mesh = mp.solutions.face_mesh

  all_scores = []
  cap = cv.VideoCapture(video_path)
  frame_cnt = 0
  # set up output file
  if save_text:
    outtext_name = os.path.basename(video_path).replace('.mp4','_output.csv')
    # folder = video_path.split('/')[-1].split(" ")[1].split("-")[0]
    folder = video_path.split('/')[-2].split("/")[0]
    path = os.path.join(output_path, folder)
    if os.path.isdir(path):
      print(path)
    else:
      os.mkdir(path)
    outtext_name = os.path.join(path, outtext_name)
    print(outtext_name)
    f = open(outtext_name, "w")

  if vis:
    outvis_name = os.path.basename(video_path).replace('.mp4','_output.avi')
    outvis_name = os.path.join(path, outvis_name)
    # We need to set resolutions.
    # so, convert them from float to integer.
    imwidth = int(cap.get(3)); imheight = int(cap.get(4))
    # Below VideoWriter object will create
    # a frame of above defined The output 
    # is stored in outvis file.
    outvid = cv.VideoWriter(outvis_name,cv.VideoWriter_fourcc(*'MJPG'), 10, (imwidth,imheight))

  
  # cap = cv.VideoCapture(1)
  with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
    while True:
      ret,frame = cap.read()
      if not ret:
        break
      frame_cnt += 1
      # frame = cv.flip(frame,1)
      rgb_frame = cv.cvtColor(frame,cv.COLOR_BGR2RGB)
      height, width, channels = frame.shape
      results = face_detection.process(frame)

      if results.detections:
        for detection in results.detections:
          results.detections[0].location_data.relative_bounding_box.xmin
          l , t = normaliz_pixel(detection.location_data.relative_bounding_box.xmin ,detection.location_data.relative_bounding_box.ymin , width, height )
          r = detection.location_data.relative_bounding_box.width*width + l
          b = detection.location_data.relative_bounding_box.height*height + t

          l -= (r-l)*0.3
          r += (r-l)*0.3
          t -= (b-t)*0.3
          b += (b-t)*0.3

          bbox = ([l,t,r,b])
        frame = Image.fromarray(frame)
        face = frame.crop(bbox)
        frame = np.asarray(face)

      with mp_face_mesh.FaceMesh(
          max_num_faces=1,
          refine_landmarks=True,
          min_detection_confidence=0.5,
          min_tracking_confidence=0.5
      )as face_mesh:
          # while True:
          #     ret,frame = cap.read()
          #     if not ret:
          #         break
          #     frame = cv.flip(frame,1)
          #     rgb_frame = cv.cvtColor(frame,cv.COLOR_BGR2RGB)
          img_h, img_w=frame.shape[:2]
          results=face_mesh.process(frame)
          if results.multi_face_landmarks:
              mesh_points =np.array([np.multiply([p.x,p.y],[img_w,img_h]).astype(int) for p in results.multi_face_landmarks[0].landmark])
              (l_cx, l_cy), l_radius = cv.minEnclosingCircle(mesh_points[LEFT_IRIS])
              (r_cx, r_cy), r_radius = cv.minEnclosingCircle(mesh_points[RIGHT_IRIS])

              center_left = np.array([l_cx, l_cy], dtype=np.int32)
              center_right = np.array([r_cx, r_cy], dtype=np.int32)
              
              cv.circle(frame, center_left, int(l_radius), (255,0,255), 1, cv.LINE_AA)
              cv.circle(frame, center_right, int(r_radius), (255,0,255), 1, cv.LINE_AA)
              cv.circle(frame, mesh_points[R_H_LEFT][0], 1, (255,255,255), -1, cv.LINE_AA)
              cv.circle(frame, mesh_points[R_H_RIGHT][0],1, (0,255,255), -1, cv.LINE_AA)
              cv.circle(frame, mesh_points[R_U_landmark][0], 1, (255,255,255), -1, cv.LINE_AA)
              cv.circle(frame, mesh_points[R_D_landmark][0],1, (0,255,255), -1, cv.LINE_AA)
              
  #             time.sleep(0.2)
  #             print("right eye: the most right",mesh_points[R_H_RIGHT][0][1])
  #             print("right eye: the most left",mesh_points[R_H_LEFT][0][1])
  #             print("right eye: the center of iris",center_right[1])
              
              iris_pos,ratio_h,ratio_v = iris_position_in_eye(center_right,mesh_points[R_H_RIGHT][0],mesh_points[R_H_LEFT][0],mesh_points[R_U_landmark][0],mesh_points[R_D_landmark][0])
  #             print(iris_pos)
              

              cv.putText(frame , f"Iris pos: {iris_pos} {ratio_h:.2f} {ratio_v:.2f} " ,(30,30),cv.FONT_HERSHEY_PLAIN,1.2,(0,255,0),1,cv.LINE_AA)
              if iris_pos == 'center':
                score = 1
              else :
                score = 0
              all_scores.append(score)
              if save_text:
                f.write("%d,%f\n"%(frame_cnt,score))
              if vis:
                  outvid.write(frame)

              
              if display:
                print(f"Iris pos: {iris_pos} {ratio_h:.2f} {ratio_v:.2f} " )
                cv2_imshow(frame)
                clear_output(wait=True)
              # cv.imshow('img',frame)

              key = cv.waitKey(1)
              if key == ord('q'):
                  break
  if save_text:
    f.write("score of eye contact: %f\n"%(average(all_scores)))
    print(video_path,"score of eye contact: %f\n"%(average(all_scores)))
    f.close()
  
  if vis:
    outvid.release()

  cap.release()
  cv.destroyAllWindows()
  return average(all_scores) ,  1 if average(all_scores)>0.5 else 0


In [ ]:
def get_videos(path_folder):
  """
  read all videos in passed path
  input:
  path_folder: the path of folder contains videos

  output:
  all_video_paths: list contains names of folders in passed folder

  """
  folders = os.listdir(path_folder)
  all_video_paths_1 = []
  all_video_paths_2 = []
  for folder in folders:
    curr_dataset_path = os.path.join(path_folder, folder)
    if os.path.isfile(curr_dataset_path):
      all_video_paths_1.append(curr_dataset_path)
    else:
      # print(curr_dataset_path)
      all_video_paths_2.extend(get_videos(curr_dataset_path))

    all_video_paths = all_video_paths_1 + all_video_paths_2
  return all_video_paths

In [ ]:
# for run and processing more than one video simutanously in videos:

videos = get_videos("/content/drive/MyDrive/dataset of presentation scoring")
for video in videos:
  ##
  ## this code to prevent execution again
  outtext_name = os.path.basename(video).replace('.mp4','_output.csv')
  folder = video.split('/')[-2].split("/")[0]
  output_path = '/content/drive/MyDrive/The results of eye contact/unsupervised'
  path = os.path.join(output_path, folder)
  outtext_name = os.path.join(path, outtext_name)
  print(outtext_name)
  if os.path.isfile(outtext_name):
    print("already is runing")
    pass
  ##
  else:
    detect_eye_contact_unsupervised (video , save_text = True , vis = False , 
         output_path = '/content/drive/MyDrive/The results of eye contact/unsupervised')



/content/drive/MyDrive/The results of eye contact/unsupervised/Presenter No1-Asylah/C_presenter No1-groupC_output.csv
already is runing
/content/drive/MyDrive/The results of eye contact/unsupervised/Presenter No1-Asylah/C_presenter No1-groupA_output.csv
already is runing
/content/drive/MyDrive/The results of eye contact/unsupervised/Presenter No1-Asylah/C_presenter No1-groupB_output.csv
already is runing
/content/drive/MyDrive/The results of eye contact/unsupervised/Asylah-GroupD/C_Presenter No1-GroupD-part1_output.csv
already is runing
/content/drive/MyDrive/The results of eye contact/unsupervised/Asylah-GroupD/C_Presenter No1-GroupD-part2_output.csv
already is runing
/content/drive/MyDrive/The results of eye contact/unsupervised/Presenter No10-Rana/C_Presenter No10-GroupA_output.csv
already is runing
/content/drive/MyDrive/The results of eye contact/unsupervised/Presenter No10-Rana/C_Presenter No10-GroupB_output.csv
already is runing
/content/drive/MyDrive/The results of eye contact/

In [ ]:
# path_file = '/content/drive/MyDrive/Eye contact data /No11/C_Presenter No11-GroupD.mp4'

# data_path = "/content/drive/MyDrive/Eye contact data "
# all_dataset = "/content/drive/MyDrive/dataset of presentation scoring"

# folders ,videos = get_videos(all_dataset)
# # print(folders)
# videos
# run('/content/drive/MyDrive/dataset of presentation scoring/Asylah-GroupD')

print(os.path.isfile('/content/drive/MyDrive/Eye contact data /No11/C_Presenter No11-GroupD.mp4'))


True


In [ ]:
# convert the results dictionary to dataframe 
df = pd.DataFrame(the_results)
import os  
os.makedirs('/content/drive/MyDrive/', exist_ok=True)  
df.to_csv('/content/drive/MyDrive/result_of_eye_contact.csv')  

0
